# Hail DB Data Validation

# `.env`

- Make sure to create a `.env` file with:
```
AZURE_ENDPOINT=...
AZURE_API_KEY=...
AZURE_MODEL=...
GCP_API_KEY=...
```
- Make sure to enable Geocoding API in your Google Cloud Platform project

# Define LLM Inference Code

In [ ]:
!pip install openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os
import re
import json
from openai import AzureOpenAI
from typing import Optional

class GPTModel:
  def __init__(self, api_key: str, endpoint: str, model: str):
    self.client = AzureOpenAI(
      azure_endpoint = endpoint,
      api_key = api_key,
      api_version = "2024-02-01"
    )
    self.model = model

  @staticmethod
  def extract_json_content(s: str) -> str:
    # Define the regex pattern to match `json<content>`
    pattern = r'^```json(.*)```$'

    # Search for the pattern
    match = re.search(pattern, s, re.DOTALL)

    # If a match is found, return the content within the wrapper
    if match:
        return match.group(1).strip()
    else:
        # Return the original string or handle as needed
        return s

  @staticmethod
  def json_to_dict(s: str) -> dict:
    return json.loads(s)

  def call(self, user_msg: str, sys_msg: Optional[str] = None) -> str:
    messages = []

    if sys_msg:
      messages.append({"role": "system", "content": sys_msg})
    messages.append({"role": "user", "content": user_msg})

    response = self.client.chat.completions.create(
        model=self.model,
        messages=messages,
    )

    res = response.choices[0].message.content
    res = GPTModel.extract_json_content(res)
    res = GPTModel.json_to_dict(res)
    return res

In [ ]:
from dotenv import load_dotenv
load_dotenv()

AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_MODEL = os.getenv('AZURE_MODEL')

gpt_model = GPTModel(
    api_key=AZURE_API_KEY,
    endpoint=AZURE_ENDPOINT,
    model=AZURE_MODEL
)

# Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Change the path as needed
df_extracted_info = pd.read_csv('/content/drive/My Drive/Hackathon_20231007/datasets/extracted_info_combined.csv')

In [ ]:
import pandas as pd

df_og = pd.read_csv('/content/drive/My Drive/Hackathon_20231007/Hail_db/hail_db_with_size_v20230204.csv')

In [ ]:
import pandas as pd

df_hail_reference_objects = pd.read_csv('/content/drive/My Drive/Hackathon_20231007/Hail_db/HailReferenceObjects.csv')

# Extract Hail *Diameter*

In [ ]:
reference_object_to_hail_diameter = dict(zip(df_hail_reference_objects['Reference Object'], df_hail_reference_objects['Hail Diameter (mm)']))

In [ ]:
import numpy as np

# Convert extracted reference object to hail size
df_extracted_info['hail_diameter'] = df_extracted_info['reference_object'].map(reference_object_to_hail_diameter)
df_extracted_info['hail_diameter'] = df_extracted_info['hail_diameter'].fillna(np.nan)

In [ ]:
df_extracted_info['hail_diameter']

0        56.0
1        12.0
2        12.0
3        12.0
4        43.0
        ...  
6995     12.0
6996     12.0
6997    123.0
6998     30.0
6999      NaN
Name: hail_diameter, Length: 7000, dtype: float64

# (1) Compare Reference Object and Hail Diameter

In [ ]:
def is_similar_size(ac_size, ob_size):
  if np.isnan(ac_size) and np.isnan(ob_size):
    return True
  elif np.isnan(ac_size) or np.isnan(ob_size):
    return False
  return ac_size * 0.75 <= ob_size <= ac_size * 1.25

reference_object_matches = []
hail_diameter_matches = []

for row_index in df_extracted_info['row_index']:
  actual_reference_object = df_og.loc[row_index, 'Reference Object']
  extracted_reference_object = df_extracted_info.loc[df_extracted_info['row_index'] == row_index, 'reference_object'].iloc[0]
  actual_hail_diameter = df_og.loc[row_index, 'Hail Diameter (mm) FINAL']
  extracted_hail_diameter = df_extracted_info.loc[df_extracted_info['row_index'] == row_index, 'hail_diameter'].iloc[0]

  if actual_reference_object == extracted_reference_object:
    reference_object_matches.append(True)
  else:
    print("=" * 10 + f" Mismatch between actual and extracted reference object at row {row_index} " + "=" * 10)
    print(f"Actual Reference Object: {actual_reference_object}")
    print(f"Extracted Reference Object: {extracted_reference_object}")
    reference_object_matches.append(False)

  if is_similar_size(actual_hail_diameter, extracted_hail_diameter):
    hail_diameter_matches.append(True)
  else:
    print("=" * 10 + f" Mismatch between actual and extracted hail diameter at row {row_index} " + "=" * 10)
    print(f"Actual Hail Diameter: {actual_hail_diameter}")
    print(f"Extracted Hail Diameter: {extracted_hail_diameter}")
    hail_diameter_matches.append(False)

df_extracted_info['reference_object_matches'] = reference_object_matches
df_extracted_info['hail_diameter_matches'] = hail_diameter_matches

Streaming output truncated to the last 5000 lines.
Actual Reference Object: golf ball
Extracted Reference Object: dime
========== Mismatch between actual and extracted hail diameter at row 595 ==========
Actual Hail Diameter: 43.0
Extracted Hail Diameter: 18.0
========== Mismatch between actual and extracted reference object at row 601 ==========
Actual Reference Object: small
Extracted Reference Object: estimated 14mm
========== Mismatch between actual and extracted hail diameter at row 601 ==========
Actual Hail Diameter: 5.0
Extracted Hail Diameter: 14.0
========== Mismatch between actual and extracted reference object at row 610 ==========
Actual Reference Object: nickel
Extracted Reference Object: pea
========== Mismatch between actual and extracted hail diameter at row 610 ==========
Actual Hail Diameter: 21.0
Extracted Hail Diameter: 12.0
========== Mismatch between actual and extracted reference object at row 611 ==========
Actual Reference Object: nickel
Extracted Reference Ob

In [ ]:
reference_object_matches_count = df_extracted_info['reference_object_matches'].value_counts()
reference_object_matches_count

reference_object_matches
True     5810
False    1190
Name: count, dtype: int64

In [ ]:
reference_object_matches_true_count = reference_object_matches_count[0]
reference_object_matches_false_count = reference_object_matches_count[1]
print(f"Reference Object Extraction Acc: {((reference_object_matches_true_count / (reference_object_matches_true_count + reference_object_matches_false_count)) * 100):.2f}%")

Reference Object Extraction Acc: 83.00%


In [ ]:
hail_diameter_matches_count = df_extracted_info['hail_diameter_matches'].value_counts()
hail_diameter_matches_count

hail_diameter_matches
True     6398
False     602
Name: count, dtype: int64

In [ ]:
hail_diameter_matches_true_count = hail_diameter_matches_count[0]
hail_diameter_matches_false_count = hail_diameter_matches_count[1]
print(f"Hail Diameter Extraction Acc: {((hail_diameter_matches_true_count / (hail_diameter_matches_true_count + hail_diameter_matches_false_count)) * 100):.2f}%")

Reference Object Extraction Acc: 91.40%


# (2) Comparing reference location with provided latitude, longitude

In [ ]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=b0711eebf079b7ae833bc974174ca7aafed90666a260d643d63795028c4bc39d
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
import googlemaps
import os
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

def get_address_from_coordinates(latitude, longitude, api_key):
    # Initialize the Google Maps client
    gmaps = googlemaps.Client(key=api_key)

    # Perform reverse geocode
    reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))

    # Extract address from the result
    if reverse_geocode_result:
        address = reverse_geocode_result[0]['formatted_address']
        return address
    else:
        return "unknown"

api_key = os.environ.get("GCP_API_KEY")
addresses = []

for row_index in tqdm(df_extracted_info['row_index']):
  try:
    latitude = df_og.loc[row_index, 'Latitude']
    longitude = df_og.loc[row_index, 'Longitude']
    address = get_address_from_coordinates(latitude, longitude, api_key)
    addresses.append(address)
  except Exception as e:
    print(e)
    print("=" * 10 + f" Could not get address at row {row_index} " + "=" * 10)
    addresses.append("unknown")
    continue

df_extracted_info['actual_reference_location'] = addresses

100%|██████████| 7000/7000 [15:12<00:00,  7.67it/s]


In [ ]:
df_extracted_info['actual_reference_location']

0                         WCJ2+22 Hope Valley, AB, Canada
1       1604 Idylwyld Dr N, Saskatoon, SK S7L 1B1, Canada
2            404 Fourth Ave, Spalding, SK S0K 4C0, Canada
3                     8GPJ+CC Strathcona Park, MB, Canada
4                             XM38+XR Priddis, AB, Canada
                              ...                        
6995                  SK-44, Mantario, SK S0L 2J0, Canada
6996                          87CF+QF Dalmeny, SK, Canada
6997                      5R93+W8 Markerville, AB, Canada
6998                         J3F2+42 Neapolis, AB, Canada
6999             5018 57 Ave #1, Olds, AB T4H 1J1, Canada
Name: actual_reference_location, Length: 7000, dtype: object

In [ ]:
from pydantic import BaseModel

class Response(BaseModel):
  reference_location_matches: str
  match_reasoning: str

response_example = Response(
    reference_location_matches="MATCHED",
    match_reasoning="Both locations are located within 10km of each other."
)

def extract_info_from_note(df, actual_reference_location, extracted_reference_location):
  sys_msg = f"Your response to the user's instruction should follow the following JSON format: {response_example.json()}"

  user_msg = f"""
Instruction:
Given the following actual reference location and extracted reference location, determine the following:
- Output MATCHED if the locations semantically match, such as having the same town name. Ignore minor differences in notations or additional details such as street names and directions
- Output CLOSE_PROXIMITY if the locations don't sementically match, but to the best of your knowledge, the two locations are in close-proximity to each other. The proximity is considered close if the two locations are roughly within 100km of each other. Use best of your knowledge.
- Output NOT_MATCHED if the locations don't semantically match and are not close-proximity, or you cannot determine if they are close-proximity.
- If the match is CLOSE_PROXIMITY, provide a reasoning why the two locations are close-proximity.
- If the match is MATCHED or NOT_MATCHED, provide a reasoning why the two locations are matched or not matched.

Actual Reference Location:
{actual_reference_location}

Extracted Reference Location:
{extracted_reference_location}
"""

  res = gpt_model.call(user_msg, sys_msg)

  return res

In [ ]:
start_index = 0
end_index = 7000
thread_max_workers = 5

In [ ]:
def process_row(row_index):
    try:
        actual_reference_location = df_extracted_info.loc[df_extracted_info['row_index'] == row_index, 'actual_reference_location'].iloc[0]
        extracted_reference_location = df_extracted_info.loc[df_extracted_info['row_index'] == row_index, 'reference_location'].iloc[0]

        if extracted_reference_location == 'unknown':
            return {
                'row_index': row_index,
                'reference_location_match': 'NOT_FOUND',
                'match_reasoning': 'N/A'
            }

        if actual_reference_location == 'unknown':
            return {
                'row_index': row_index,
                'reference_location_match': 'INVALID_COORDINATES',
                'match_reasoning': 'N/A'
            }

        res = extract_info_from_note(df_og, actual_reference_location, extracted_reference_location)
        return {
            'row_index': row_index,
            'reference_location_match': res.get('reference_location_matches', 'GPT_ERROR'),
            'match_reasoning': res.get('match_reasoning', 'N/A'),
        }

    except Exception as e:
        print(f"Error occurred at row {row_index}: {e}")
        return {
            'row_index': row_index,
            'reference_location_match': 'GPT_ERROR',
            'match_reasoning': 'N/A',
        }

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

location_matches = []

# Adjust the number of workers based on your environment and task

row_indexes = df_extracted_info['row_index'][start_index:end_index].tolist()

with ThreadPoolExecutor(max_workers=thread_max_workers) as executor:
    # Submit tasks to the executor
    future_to_row = {executor.submit(process_row, row_index): row_index for row_index in row_indexes}

    # Iterate over the futures as they complete (maintain order if necessary)
    for future in tqdm(as_completed(future_to_row), total=len(future_to_row)):
        result = future.result()
        location_matches.append(result)

  2%|▏         | 123/7000 [00:48<51:09,  2.24it/s]

Error occurred at row 124: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


  2%|▎         | 175/7000 [01:34<49:26,  2.30it/s]  

Error occurred at row 173: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


  3%|▎         | 179/7000 [01:38<1:48:21,  1.05it/s]

...

 99%|█████████▉| 6923/7000 [1:32:35<01:10,  1.09it/s]

Error occurred at row 6924: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


 99%|█████████▉| 6964/7000 [1:33:09<00:14,  2.51it/s]

Error occurred at row 6962: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}
Error occurred at row 6963: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


100%|██████████| 7000/7000 [1:33:46<00:00,  1.24it/s]


In [ ]:
# First three items from locations that are matched, adjust as needed
location_matches[:5]

[{'row_index': 0,
  'reference_location_match': 'NOT_FOUND',
  'match_reasoning': 'N/A'},
 {'row_index': 1,
  'reference_location_match': 'NOT_FOUND',
  'match_reasoning': 'N/A'},
 {'row_index': 2,
  'reference_location_match': 'MATCHED',
  'match_reasoning': 'Both locations refer to Saskatchewan (SK), Canada.'},
 {'row_index': 4,
  'reference_location_match': 'CLOSE_PROXIMITY',
  'match_reasoning': 'Both locations are in the Alberta province of Canada and are within 100km of each other.'},
 {'row_index': 8,
  'reference_location_match': 'NOT_FOUND',
  'match_reasoning': 'N/A'}]

In [ ]:
df_extracted_info['reference_location_matches'] = 'GPT_ERROR'
df_extracted_info['match_reasoning'] = 'N/A'

In [ ]:
for location_match in location_matches:
    # Find the row in df_extracted_info where 'row_index' matches match['row_index']
    # and update 'reference_location_matches' and 'match_reasoning'
    df_extracted_info.loc[df_extracted_info['row_index'] == location_match['row_index'], 'reference_location_matches'] = location_match['reference_location_match']
    df_extracted_info.loc[df_extracted_info['row_index'] == location_match['row_index'], 'match_reasoning'] = location_match['match_reasoning']

In [ ]:
reference_location_matches_count = df_extracted_info['reference_location_matches'].value_counts()
reference_location_matches_count

reference_location_matches
MATCHED            3915
NOT_FOUND          1605
CLOSE_PROXIMITY     896
NOT_MATCHED         360
GPT_ERROR           224
Name: count, dtype: int64

In [ ]:
location_matched = reference_location_matches_count[0]
location_close_proximity = reference_location_matches_count[2]
location_not_matched = reference_location_matches_count[3]
print(f"Location Matched: {((location_matched + location_close_proximity) / (location_matched + location_close_proximity + location_not_matched)) * 100:.2f}%")

Location Matched: 93.04%


# (3) Comparing estimated start time with provided start time

In [ ]:
from datetime import datetime, timedelta

def is_within_three_hours(ac_t, ob_t):
    # Calculate the absolute difference between the two datetime objects
    time_difference = abs(ob_t - ac_t)

    return time_difference <= timedelta(hours=3)

start_time_matches = []

for row_index in df_extracted_info['row_index']:
  actual_start_time = df_og.loc[row_index, 'Start Time']
  extracted_start_time = df_extracted_info.loc[df_extracted_info['row_index'] == row_index, 'hail_start_time'].iloc[0]

  if extracted_start_time == 'unknown':
    start_time_matches.append('NOT_FOUND')
    continue

  try:
    actual_start_time_converted = datetime.strptime(actual_start_time, "%Y-%m-%d %H:%M:%S")
    extracted_start_time_converted = datetime.strptime(extracted_start_time, "%Y-%m-%d %H:%M:%S")
    matches = is_within_three_hours(actual_start_time_converted, extracted_start_time_converted)
    if matches:
      start_time_matches.append('MATCHED')
    else:
      print("=" * 10 + f" Mismatch between actual and extracted start time at row {row_index} " + "=" * 10)
      print(f"Actual Start Time: {actual_start_time}")
      print(f"Extracted Start Time: {extracted_start_time}")
      start_time_matches.append('NOT_MATCHED')
  except:
    print("=" * 10 + f" Could not convert start time at row {row_index} " + "=" * 10)
    print(f"Actual Start Time: {actual_start_time}")
    print(f"Extracted Start Time: {extracted_start_time}")
    start_time_matches.append('INVALID_FORMAT')
    continue

df_extracted_info['start_time_matches'] = start_time_matches

========== Mismatch between actual and extracted start time at row 30 ==========
Actual Start Time: 2005-06-01 22:00:00
Extracted Start Time: 2005-06-01 17:00:00
========== Mismatch between actual and extracted start time at row 32 ==========
Actual Start Time: 2005-07-03 01:20:00
Extracted Start Time: 2005-07-02 20:20:00
========== Mismatch between actual and extracted start time at row 41 ==========
Actual Start Time: 2006-05-28 08:00:00
Extracted Start Time: 2023-01-01 08:00:00
========== Mismatch between actual and extracted start time at row 98 ==========
Actual Start Time: 2007-05-12 19:57:00
Extracted Start Time: 2023-01-01 19:57:00
========== Mismatch between actual and extracted start time at row 774 ==========
Actual Start Time: 2006-08-09 01:40:00
Extracted Start Time: 2023-01-01 01:40:00
========== Mismatch between actual and extracted start time at row 776 ==========
Actual Start Time: 2006-08-09 03:55:00
Extracted Start Time: 2023-01-01 03:55:00
========== Mismatch betwee

In [ ]:
start_time_matches_count = df_extracted_info['start_time_matches'].value_counts()
start_time_matches_count

start_time_matches
NOT_FOUND         6954
NOT_MATCHED         40
MATCHED              3
INVALID_FORMAT       3
Name: count, dtype: int64

In [ ]:
start_time_not_matched = start_time_matches_count[1]
start_time_matched = start_time_matches_count[2]
print(f"Start Time Matched: {((start_time_matched) / (start_time_matched + start_time_not_matched)) * 100:.2f}%")

Start Time Matched: 6.98%


# Download Data

In [ ]:
from google.colab import files

df_extracted_info.to_csv('extracted_info_with_validation.csv', index=False)
files.download('extracted_info_with_validation.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# (4) Testing/archiving links to social media posts (Doesn't Work Well!!!)

In [ ]:
!pip install google-colab-selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
import requests

def get_link_status(url):
    try:
        if url[:8] != 'https://':
          url = 'https://' + url

        response = requests.head(url, allow_redirects=True)
        print(url, "->", response.status_code)
        if 200 <= response.status_code < 300:
            return 'VALID'
        elif response.status_code == 404:
            return 'INVALID'
        else:
            return 'VALID_BUT_NOT_RETRIEVABLE'
    except requests.RequestException as e:
        # In case of any requests-related exceptions, return False
        print(f"For url {url}, error {e} occured")
        return 'ERROR'

In [ ]:
import google_colab_selenium as gs
import os

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")

create_folder_if_not_exists('/content/images')

social_media_status_results = []

driver = gs.Chrome()

for row_index in df_extracted_info['row_index']:
  extracted_social_media_link = df_extracted_info.loc[df_extracted_info['row_index'] == row_index, 'social_media_link'].iloc[0]

  try:
    if extracted_social_media_link == 'unknown':
      result = {
          'row_index': row_index,
          'status': 'NOT_FOUND',
      }
      social_media_status_results.append(result)
      continue

    link_status = get_link_status(extracted_social_media_link)

    result = {
        'row_index': row_index,
        'status': link_status,
    }

    social_media_status_results.append(result)

    if link_status == 'VALID_BUT_NOT_RETRIEVABLE':
      print("=" * 10 + f" Valid but not retrievable link at row {row_index} " + "=" * 10)
      continue

    elif link_status == 'INVALID':
      print("=" * 10 + f" Invalid link at row {row_index} " + "=" * 10)
      continue

    elif link_status == 'ERROR':
      print("=" * 10 + f" Error while checking link at row {row_index} " + "=" * 10)
      continue

    driver.get(extracted_social_media_link)
    driver.save_screenshot(f'/content/images/{row_index}.png')
    print("=" * 10 + f" Screenshot saved at row {row_index} " + "=" * 10)
    result = {
        'row_index': row_index,
        'status': 'SAVED',
    }
    social_media_status_results.append(result)
    driver.quit()
  except:
    print("=" * 10 + f" Could not save screenshot at row {row_index} " + "=" * 10)
    result = {
        'row_index': row_index,
        'status': 'ERROR_WHILE_SAVING',
    }
    social_media_status_results.append(result)
    continue

Folder created: /content/images


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

https://twitter.com/aaronjayjack/status/1007418300341587970 -> 403
========== Valid but not retrievable link at row 10 ==========
https://twitter.com/UnruhAdam/status/1277042413056389120 -> 403
========== Valid but not retrievable link at row 18 ==========
https://www.twitter.com -> 403
========== Valid but not retrievable link at row 21 ==========
https://twitter.com/EMB428/status/1554276829531291648?s=20&t=P9_R_G6P4gZs7KACI_T_bw -> 403
========== Valid but not retrievable link at row 23 ==========
https://twitter.com/96blueicebunny/status/1554288164742696960?s=20&t=P9_R_G6P4gZs7KACI_T_bw, https://twitter.com/LeideniusShaye/status/1554307738435403776?s=20&t=P9_R_G6P4gZs7KACI_T_bw -> 403
========== Valid but not retrievable link at row 24 ==========
https://www.twitter.com/itWlQjzqBr -> 403
========== Valid but not retrievable link at row 226 ==========
https://www.twitter.com/JaxnZvtBpC -> 403
========== Valid but not retrievable link at row 247 ==========
https://www.youtube.com -> 2

...

https://www.facebook.com/groups/albertastorm/permalink/614032930109350/ -> 200
========== Could not save screenshot at row 6986 ==========


https://www.facebook.com/groups/605744407196622/permalink/735283230909405/ -> 200
========== Could not save screenshot at row 6989 ==========


https://www.facebook.com/groups/760797907396212/permalink/2556425837833401/ -> 200
========== Could not save screenshot at row 6990 ==========


https://www.facebook.com/100008266606501/videos/2911395642498887/ -> 200
========== Could not save screenshot at row 6991 ==========
https://twitter.com/FidaKfida/status/1563614667419054087?s=20&t=iY4g_5QTFVC3RPhLpDzsyQ https://twitter.com/ArieusK/status/1563620013986422785?s=20&t=iY4g_5QTFVC3RPhLpDzsyQ https://twitter.com/adolwyn/status/1563618996037234688?s=20&t=iY4g_5QTFVC3RPhLpDzsyQ https://twitter.com/Minseok10249565/status/1563618662996934657?s=20&t=iY4g_5QTFVC3RPhLpDzsyQ https://twitter.com/asuka194/status/1563618472223207425?s=20&t=iY4g_5QTFVC3RPhLpDzsyQ https://www.facebook.com/groups/albertastorm/permalink/616976699814973/ https://twitter.com/CalgaryRASC/status/1563626289797689344?s=20&t=MjHtPQwxmcriL6TH8R1_HA https://twitter.com/ArieusK/status/1563626958864674816?s=20&t=MjHtPQwxmcriL6TH8R1_HA -> 403
========== Valid but not retrievable link at row 6992 ==========
https://twitter.com/MitchellGrafWX/status/1563615592426643457?s=20&t=iY4g_5QTFVC3RPhLpDzsyQ -> 403
========== Val

https://www.facebook.com/groups/SaskWT/permalink/5821194641224518/ -> 200
========== Could not save screenshot at row 6996 ==========
https://www.twitter.com/BraydonMoreSo/status/1554252578719801345?s=20&t=P9_R_G6P4gZs7KACI_T_bw -> 403
========== Valid but not retrievable link at row 6997 ==========
https://twitter.com/lightningmanAB/status/1548478798059806720?s=20&t=DXNtwKELNaqVpGveIhhTvA -> 403
========== Valid but not retrievable link at row 6998 ==========
https://www.twitter.com -> 403
========== Valid but not retrievable link at row 6999 ==========


In [ ]:
df_extracted_info['social_media_link_status'] = "NOT_FOUND"

In [ ]:
for social_media_status_result in social_media_status_results:
    # Find the row in df_extracted_info where 'row_index' matches match['row_index']
    # and update 'reference_location_matches' and 'match_reasoning'
    df_extracted_info.loc[df_extracted_info['row_index'] == social_media_status_result['row_index'], 'social_media_link_status'] = social_media_status_result['status']

In [ ]:
social_media_link_status_count = df_extracted_info['social_media_link_status'].value_counts()
social_media_link_status_count

social_media_link_status
NOT_FOUND                    5924
VALID_BUT_NOT_RETRIEVABLE     635
ERROR_WHILE_SAVING            398
INVALID                        29
ERROR                          13
SAVED                           1
Name: count, dtype: int64

In [ ]:
# Download snipped social media images
import shutil
from google.colab import files

def zip_folder(folder_path, output_filename):
    shutil.make_archive(output_filename, 'zip', folder_path)
    print(f"Folder zipped into {output_filename}.zip")

# Example usage
zip_folder('/content/images', 'social_media_weather_report_images')

files.download('social_media_weather_report_images.zip')

Folder zipped into social_media_weather_report_images.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### !!!Important: to access Twitter posts, you need "Elevated" Twitter dev account